In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

In [ ]:
from state_machine import StateMachine, State


class LockedState(State):
    """
    The state which indicates that there are limited device capabilities.
    """
    def __init__(self, id: str = "Locked"):
        super().__init__(id)

    def on_event(self, event: str) -> State:
        if event == 'pin_entered':
            return UnlockedState()

        return self


class UnlockedState(State):
    """
    The state which indicates that there are no limitations on device
    capabilities.
    """
    def __init__(self, id: str = "Unlocked"):
        super().__init__(id)

    def on_event(self, event: str) -> State:
        # 
        if event == 'device_locked':
            return LockedState()

        return self
    
locked = LockedState(id='xyz')
unlocked = UnlockedState()

machine = (
  StateMachine(locked)
    .add_transition(locked, 'pin_entered', unlocked)
    .add_transition(unlocked, 'device_locked', locked)
)

machine.state
machine.on_event('pin_entered')
machine.state

# class SimpleDevice(object):
#     """ 
#     A simple state machine that mimics the functionality of a device from a 
#     high level.
#     """

#     def __init__(self):
#         """ Initialize the components. """

#         # Start with a default state.
#         self.state = LockedState()

#     def on_event(self, event: str):
#         """
#         This is the bread and butter of the state machine. Incoming events are
#         delegated to the given states which then handle the event. The result is
#         then assigned as the new state.
#         """

#         # The next state will be the result of the on_event function.
#         self.state = self.state.on_event(event)

# device = SimpleDevice()
# device.on_event('device_locked')
# device.on_event('pin_entered')
# device.state

In [ ]:
from utils import tool, Tool

# Example usage
@tool
def example_function(param1: int, param2: str) -> None:
    """
    This is an example function.
    
    Args:
        param1 (int): The first parameter.
        param2 (str): The second parameter.
    """
    print(f"param1: {param1}, param2: {param2}")

# Accessing the metadata
# print(example_function.name)

x: list[Tool] = []
x.append(example_function)

for tool in x:
    print(tool.name)
    print(tool.parameters)
    print(tool.description)

In [1]:
from messages import UserMessage
from prompt import PromptTemplate
from runner import Runner
from tools import tool, Toolbox
from typing import Literal

@tool
def get_current_weather(location: str, format: Literal["celsius", "fahrenheit"] = "celsius", days: int = 1, detailed: bool = False):
    """
    Get the current weather.
    
    location: The city and state, e.g. San Francisco, CA
    format: The temperature unit to use. Infer this from the user's location.
    days: Number of days to forecast.
    detailed: Whether to include detailed information.
    """
    pass


@tool
def post_slack(question: str, channel: str):
    """
    Post to a slack channel.

    Args:
        question: The question to post.
        channel: The channel to post to.
    """
    print(f"Posted '{question}' to '{channel}'")

@tool
def get_jira(team: str, ticket: str):
    """
    Get a jira ticket based on the team and ticket number.

    Args:
        team: The team to search for the ticket.
        ticket: The ticket number to search for.
    """
    print(f"Got Jira ticket '{ticket}' for team '{team}'")



toolbox = (
    Toolbox()
        .add_tool(post_slack)
        .add_tool(get_jira)
)

# t = toolbox.get_tool("post_slack")
# t(question="Hello", channel="general")

# for t in toolbox.tools:
#     print(t.schema)

import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = "gpt-4o-mini"

template = PromptTemplate(template="me the jira ticket {ticket_num} for my team")
prompt = template.render(ticket_num=432)

runner = (
    Runner()
      .add_llm(client, model)
      .add_system_message("You are a helpful assistant.")
      .add_history([
          UserMessage("I work on team combo")
      ])
      .add_user_message(prompt)
      .add_tools(toolbox)
)
runner.run()



Got Jira ticket '432' for team 'combo'


[None]